# Master Data Management (MDM) - BigQuery Native Batch Processing

This notebook demonstrates a complete end-to-end Master Data Management pipeline using BigQuery's native capabilities:

- **Data Generation**: Create realistic sample data with duplicates and variations
- **Data Ingestion**: Load data into BigQuery from multiple sources
- **Data Standardization**: Clean and normalize data using SQL
- **Embedding Generation**: Use BigQuery ML with `gemini-embedding-001`
- **Vector Indexing**: Create vector indexes for fast similarity search
- **Entity Matching**: Implement exact, fuzzy, vector, business rules, and AI natural language matching
- **Confidence Scoring**: Calculate match confidence and make decisions
- **Golden Record Creation**: Generate master entities with survivorship rules
- **Analysis & Visualization**: Analyze results and performance

## Architecture Overview

This implementation follows the batch processing path from the MDM architecture:
1. **Files/APIs/Databases** → **BigQuery Raw Tables**
2. **BigQuery Standardization** → **BigQuery Staging**
3. **BigQuery ML Embeddings** → **BigQuery with Embeddings**
4. **BigQuery Vector Search** → **Unified Matching Engine**
5. **Confidence Scoring** → **Golden Record Creation**
6. **Master Entities** → **Analytics & Distribution**

## 1. Setup and Configuration

In [1]:
# Import required libraries

from bigquery_utils import (
    BigQueryMDMHelper,
    generate_standardization_sql,
    generate_union_sql,
    generate_embedding_sql,
    generate_exact_matching_sql,
    generate_fuzzy_matching_sql,
    generate_vector_matching_sql,
    generate_business_rules_sql,
    generate_combined_scoring_sql,
    generate_ai_natural_language_matching_sql,
    generate_golden_record_sql
)

from data_generator import MDMDataGenerator

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.cloud import bigquery
from google.auth import default
import warnings

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [ ]:
# Configuration
PROJECT_ID = "your-project-id"  # Replace with your GCP project ID
DATASET_ID = "mdm_demo"
LOCATION = "US"

# Initialize BigQuery helper
try:
    bq_helper = BigQueryMDMHelper(PROJECT_ID, DATASET_ID)
    print(f"✅ Connected to BigQuery project: {PROJECT_ID}")
    print(f"📊 Dataset: {bq_helper.dataset_ref}")
except Exception as e:
    print(f"❌ Error connecting to BigQuery: {e}")
    print("Please ensure you have:")
    print("1. Set up Google Cloud authentication")
    print("2. Enabled BigQuery API")
    print("3. Updated PROJECT_ID above")

✅ Connected to BigQuery project: johanesa-playground-326616
📊 Dataset: johanesa-playground-326616.mdm_demo


## 2. Generate Sample Data

Create realistic customer data from multiple sources with intentional duplicates and variations.

In [3]:
# Generate sample data
print("🔄 Generating sample customer data...")
generator = MDMDataGenerator(num_unique_customers=120)
datasets = generator.generate_all_datasets()

# Display summary statistics
print("\n📈 Dataset Summary:")
total_records = 0
for source, df in datasets.items():
    print(f"  {source.upper()}: {len(df):,} records")
    total_records += len(df)

print(f"\n📊 Total records: {total_records:,}")
print(f"👥 Unique customers: {generator.num_unique_customers:,}")
print(
    f"🔄 Duplication factor: {total_records / generator.num_unique_customers:.2f}x")

# Show sample records from each source
print("\n🔍 Sample Records:")
for source, df in datasets.items():
    print(f"\n{source.upper()} Sample:")
    display(df[['record_id', 'full_name', 'email',
            'phone', 'address', 'source_system']].head(3))

🔄 Generating sample customer data...

📈 Dataset Summary:
  CRM: 105 records
  ERP: 84 records
  ECOMMERCE: 95 records

📊 Total records: 284
👥 Unique customers: 120
🔄 Duplication factor: 2.37x

🔍 Sample Records:

CRM Sample:


,record_id,full_name,email,phone,address,source_system
0,13a62d92-16dd-4829-909f-5e962a22ef72,Nicholas Gray,blacknicholas@example.org,665.557.7908,25913 Shepherd Stravenue,crm
1,6716dc18-d47d-485b-8d6a-bccdb60eaab8,Diane Escobar,mathewaguilar@outlook.com,(346)829.148,54235 Lee Dam Apt. 141,crm
2,8a85f401-b2f2-456e-bb51-9ed3ea98cf6d,James Herrera,robertramirez@example.org,478-810-8013,3602 Smith Loaf Suite 746,crm



ERP Sample:


,record_id,full_name,email,phone,address,source_system
0,811d9b3c-16c6-4403-a009-c66c0c81c554,Jeffrey Black,finleycasey@gmail.com,545-918-7053,738 Edward Lodge Apt. 385,erp
1,695227a5-8cd5-4254-9331-ff5af7c1fa7c,Elaine Nelson,robertroach@example.net,001-791-841-,08737 Young Bridge Apt. 197,erp
2,6cec54c1-7da7-4b9b-abdd-d41b428a9759,Caleb Joseph,crawfordwilliam@example.org,292.552.4191,5622 Angela Station,erp



ECOMMERCE Sample:


,record_id,full_name,email,phone,address,source_system
0,95ce6cdc-d285-4c83-9c20-70ec8a73b370,Leslie Villanueva,travis84@example.com,792.808.6780,1875 Alexander Fords,ecommerce
1,cefe2cc7-42bc-4e25-8e20-205ace0ad4d6,Michael Nichols,john16@example.net,001-951-582-,343 James Gateway,ecommerce
2,722acb4b-5226-4b0f-9e52-30926b85d34a,Troy Ward,tanner82@example.net,(314) 467-8669,177 Anderson Village Apt. 268,ecommerce


## 3. Data Ingestion to BigQuery

Load the generated data into BigQuery raw tables.

In [4]:
# Create dataset
print("🔄 Creating BigQuery dataset...")
bq_helper.create_dataset()

# Load data to BigQuery
print("\n🔄 Loading data to BigQuery...")
for source, df in datasets.items():
    table_name = f"raw_{source}_customers"
    print(f"  Loading {source} data to {table_name}...")
    bq_helper.load_dataframe_to_table(df, table_name)

print("\n✅ Data ingestion completed!")

# Verify data loading
print("\n📊 Table Information:")
for source in datasets.keys():
    table_name = f"raw_{source}_customers"
    info = bq_helper.get_table_info(table_name)
    if info:
        print(
            f"  {table_name}: {info['num_rows']:,} rows, {info['num_bytes']:,} bytes")

🔄 Creating BigQuery dataset...
Dataset johanesa-playground-326616.mdm_demo created or already exists

🔄 Loading data to BigQuery...
  Loading crm data to raw_crm_customers...
Loaded 105 rows to johanesa-playground-326616.mdm_demo.raw_crm_customers
  Loading erp data to raw_erp_customers...
Loaded 84 rows to johanesa-playground-326616.mdm_demo.raw_erp_customers
  Loading ecommerce data to raw_ecommerce_customers...
Loaded 95 rows to johanesa-playground-326616.mdm_demo.raw_ecommerce_customers

✅ Data ingestion completed!

📊 Table Information:
  raw_crm_customers: 105 rows, 31,826 bytes
  raw_erp_customers: 84 rows, 25,294 bytes
  raw_ecommerce_customers: 95 rows, 29,379 bytes


## 4. Data Standardization

Clean and standardize data from all sources using BigQuery SQL.

In [5]:
# Combine all raw data into a single table
print("🔄 Combining raw data from all sources...")

combine_sql = generate_union_sql(bq_helper.dataset_ref)

bq_helper.execute_query(combine_sql)
print("✅ Raw data combined")

# Standardize the combined data
print("\n🔄 Standardizing data...")
standardization_sql = generate_standardization_sql(
    f"{bq_helper.dataset_ref}.raw_customers_combined",
    f"{bq_helper.dataset_ref}.customers_standardized"
)

bq_helper.execute_query(standardization_sql)
print("✅ Data standardization completed")

# Show standardization results
sample_query = f"""
SELECT
  record_id,
  source_system,
  full_name,
  full_name_clean,
  email,
  email_clean,
  phone,
  phone_clean,
  address,
  address_clean
FROM `{bq_helper.dataset_ref}.customers_standardized`
LIMIT 5
"""

sample_df = bq_helper.execute_query(sample_query)
print("\n🔍 Standardization Sample:")
display(sample_df)

🔄 Combining raw data from all sources...
✅ Raw data combined

🔄 Standardizing data...
✅ Data standardization completed

🔍 Standardization Sample:


,record_id,source_system,full_name,full_name_clean,email,email_clean,phone,phone_clean,address,address_clean
0,578cb827-4ec2-4c1c-9f2c-2c34605cb8da,crm,Heidi Spencer,HEIDI SPENCER,robertbentley@hotmail.com,robertbentley@hotmail.com,(001) 729-994-,001729994,3872 Justin Shore Ste 134,3872 JUSTIN SHORE STE 134
1,ed99b3f4-d358-4943-9b91-070340c2ec8c,crm,Heidi Spencer,HEIDI SPENCER,robertbentley@example.net,robertbentley@example.net,001-729-994-,001729994,3872 Justin Shore Suite 134,3872 JUSTIN SHORE SUITE 134
2,747a04ff-1776-4c39-a978-3f51cc9ef6cc,erp,Heidi Spencer,HEIDI SPENCER,robertbentley@example.net,robertbentley@example.net,001-729-994-,001729994,3872 Justin Shore Suite 134,3872 JUSTIN SHORE SUITE 134
3,560818f9-ed1d-471f-849a-d4c6ad5e8f05,erp,Jeffrey White,JEFFREY WHITE,hartmanerik@outlook.com,hartmanerik@outlook.com,+1-661-553-3,16615533,015 Allen Row Apt. 056,015 ALLEN ROW APT. 056
4,3919d7e8-eb40-4dea-8bc2-14458e3b9d53,ecommerce,Jeffrey White,JEFFREY WHITE,hartmanerik@example.net,hartmanerik@example.net,+1-661-553-3,16615533,015 Allen Row Apt. 056,015 ALLEN ROW APT. 056


## 5. Embedding Generation with BigQuery ML

Generate embeddings using BigQuery's native ML.GENERATE_EMBEDDING function with the latest `gemini-embedding-001` model.

In [6]:
# Create embedding model
print("\n🔄 Creating embedding model...")
model_sql = f"""
CREATE OR REPLACE MODEL `{bq_helper.dataset_ref}.embedding_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS(
  ENDPOINT = 'gemini-embedding-001'
)
"""

try:
    bq_helper.execute_query(model_sql)
    print("✅ Embedding model created successfully")
except Exception as e:
    print(f"❌ Error creating model: {e}")
    print("Please ensure:")
    print("1. You have necessary permissions")
    print("2. Vertex AI API is enabled")


🔄 Creating embedding model...
✅ Embedding model created successfully


In [7]:
# Generate embeddings
print("🔄 Generating embeddings...")
embedding_sql = generate_embedding_sql(
    f"{bq_helper.dataset_ref}.customers_standardized",
    f"{bq_helper.dataset_ref}.customers_with_embeddings",
    f"{bq_helper.dataset_ref}.embedding_model"
)

try:
    bq_helper.execute_query(embedding_sql)
    print("✅ Embeddings generated successfully")

    # Check embedding dimensions
    check_sql = f"""
    SELECT
      COUNT(*) as total_records,
      COUNT(ml_generate_embedding_result) as records_with_embeddings,
      ANY_VALUE(ARRAY_LENGTH(ml_generate_embedding_result)) AS embedding_dimension
    FROM `{bq_helper.dataset_ref}.customers_with_embeddings`
    WHERE ml_generate_embedding_result IS NOT NULL
    LIMIT 1
    """

    result = bq_helper.execute_query(check_sql)
    if not result.empty:
        print(f"📊 Embedding Statistics:")
        print(f"  Total records: {result.iloc[0]['total_records']:,}")
        print(
            f"  Records with embeddings: {result.iloc[0]['records_with_embeddings']:,}")
        print(
            f"  Embedding dimension: {result.iloc[0]['embedding_dimension']}")

except Exception as e:
    print(f"❌ Error generating embeddings: {e}")
    print("This might be due to:")
    print("1. Insufficient permissions")
    print("2. API quotas or limits")

🔄 Generating embeddings...
✅ Embeddings generated successfully
📊 Embedding Statistics:
  Total records: 284
  Records with embeddings: 284
  Embedding dimension: 3072


## 6. Vector Index Creation

Create vector indexes for efficient similarity search.

In [8]:
# Note: Vector index creation (with IVF) requires minimum 5,000 rows
# For our sample dataset, we'll use direct vector search
# which is actually more efficient for small datasets

# Create vector index for fast similarity search (will results an error)
print("🔄 Creating vector index...")

vector_index_sql = f"""
CREATE VECTOR INDEX IF NOT EXISTS customer_embedding_index
ON `{bq_helper.dataset_ref}.customers_with_embeddings`(ml_generate_embedding_result)
OPTIONS(
  index_type = 'IVF',
  distance_type = 'COSINE'
)
"""

try:
    bq_helper.execute_query(vector_index_sql)
    print("✅ Vector index created successfully")
    print("📈 This will significantly speed up vector similarity searches")
except Exception as e:
    print(f"⚠️ Vector index creation failed: {e}")
    print("Vector search will still work but may be slower")
    print("Vector indexes require specific BigQuery editions and regions")

🔄 Creating vector index...
Error executing query: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/johanesa-playground-326616/queries/a642acc9-b1d9-48f0-9c2e-01c18d7f2c1b?maxResults=0&location=US&prettyPrint=false: Total rows 284 is smaller than min allowed 5000 for CREATE VECTOR INDEX query with the IVF index type. Please use VECTOR_SEARCH table-valued function directly to perform the similarity search.

Location: US
Job ID: a642acc9-b1d9-48f0-9c2e-01c18d7f2c1b
 [{'@type': 'type.googleapis.com/google.rpc.DebugInfo', 'detail': '[INVALID_INPUT] message=QUERY_ERROR: [Total rows 284 is smaller than min allowed 5000 for CREATE VECTOR INDEX query with the IVF index type. Please use VECTOR_SEARCH table-valued function directly to perform the similarity search.] debug=code: \t BAD_QUERY\ndescription: "Total rows 284 is smaller than min allowed 5000 for CREATE VECTOR INDEX query with the IVF index type. Please use VECTOR_SEARCH table-valued function directly to perform the similari

## 7. Entity Matching

Implement multiple matching strategies using BigQuery SQL:
- **Exact Matching**: Direct field comparison
- **Fuzzy Matching**: String similarity algorithms
- **Vector Matching**: Semantic similarity using embeddings
- **Business Rules**: Domain-specific logic
- **AI Natural Language**: Direct AI comparison using Gemini 2.5 Pro

In [9]:
# 7.1 Exact Matching
print("🔄 Running exact matching...")
exact_sql = generate_exact_matching_sql(
    f"{bq_helper.dataset_ref}.customers_with_embeddings")
bq_helper.execute_query(exact_sql)
print("✅ Exact matching completed")

# Check exact match results
exact_count_sql = f"""
SELECT
  COUNT(*) as total_exact_matches,
  COUNT(CASE WHEN email_exact_score > 0 THEN 1 END) as email_matches,
  COUNT(CASE WHEN phone_exact_score > 0 THEN 1 END) as phone_matches,
  COUNT(CASE WHEN id_exact_score > 0 THEN 1 END) as id_matches
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_exact_matches`
"""

exact_stats = bq_helper.execute_query(exact_count_sql)
print(
    f"📊 Exact Match Results: {exact_stats.iloc[0]['total_exact_matches']} total matches")
print(f"  📧 Email matches: {exact_stats.iloc[0]['email_matches']}")
print(f"  📞 Phone matches: {exact_stats.iloc[0]['phone_matches']}")
print(f"  🆔 ID matches: {exact_stats.iloc[0]['id_matches']}")

🔄 Running exact matching...
✅ Exact matching completed
📊 Exact Match Results: 258 total matches
  📧 Email matches: 153
  📞 Phone matches: 233
  🆔 ID matches: 258


In [10]:
# 7.2 Fuzzy Matching
print("🔄 Running fuzzy matching...")
fuzzy_sql = generate_fuzzy_matching_sql(
    f"{bq_helper.dataset_ref}.customers_with_embeddings")
bq_helper.execute_query(fuzzy_sql)
print("✅ Fuzzy matching completed")

# Check fuzzy match results
fuzzy_count_sql = f"""
SELECT
  COUNT(*) as total_fuzzy_matches,
  AVG(name_fuzzy_score) as avg_name_score,
  AVG(address_fuzzy_score) as avg_address_score,
  AVG(fuzzy_overall_score) as avg_overall_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_fuzzy_matches`
"""

fuzzy_stats = bq_helper.execute_query(fuzzy_count_sql)
print(
    f"📊 Fuzzy Match Results: {fuzzy_stats.iloc[0]['total_fuzzy_matches']} total matches")
print(f"  👤 Avg name score: {fuzzy_stats.iloc[0]['avg_name_score']:.3f}")
print(f"  🏠 Avg address score: {fuzzy_stats.iloc[0]['avg_address_score']:.3f}")
print(f"  📈 Avg overall score: {fuzzy_stats.iloc[0]['avg_overall_score']:.3f}")

🔄 Running fuzzy matching...
✅ Fuzzy matching completed
📊 Fuzzy Match Results: 890.0 total matches
  👤 Avg name score: 0.695
  🏠 Avg address score: 0.496
  📈 Avg overall score: 0.596


In [11]:
# 7.3 Vector Matching
print("🔄 Running vector similarity matching...")
vector_sql = generate_vector_matching_sql(
    f"{bq_helper.dataset_ref}.customers_with_embeddings")

try:
    bq_helper.execute_query(vector_sql)
    print("✅ Vector matching completed")

    # Check vector match results
    vector_count_sql = f"""
    SELECT
      COUNT(*) as total_vector_matches,
      AVG(vector_similarity_score) as avg_similarity,
      MIN(vector_similarity_score) as min_similarity,
      MAX(vector_similarity_score) as max_similarity
    FROM `{bq_helper.dataset_ref}.customers_with_embeddings_vector_matches`
    """

    vector_stats = bq_helper.execute_query(vector_count_sql)
    print(
        f"📊 Vector Match Results: {vector_stats.iloc[0]['total_vector_matches']} total matches")
    print(f"  📈 Avg similarity: {vector_stats.iloc[0]['avg_similarity']:.3f}")
    print(f"  📉 Min similarity: {vector_stats.iloc[0]['min_similarity']:.3f}")
    print(f"  📈 Max similarity: {vector_stats.iloc[0]['max_similarity']:.3f}")

except Exception as e:
    print(f"⚠️ Vector matching failed: {e}")
    print("This might be due to missing embeddings or vector index issues")

🔄 Running vector similarity matching...
✅ Vector matching completed
📊 Vector Match Results: 40186.0 total matches
  📈 Avg similarity: 0.801
  📉 Min similarity: 0.706
  📈 Max similarity: 1.000


In [12]:
# 7.4 Business Rules Matching
print("🔄 Running business rules matching...")
business_sql = generate_business_rules_sql(
    f"{bq_helper.dataset_ref}.customers_with_embeddings")
bq_helper.execute_query(business_sql)
print("✅ Business rules matching completed")

# Check business rules results
business_count_sql = f"""
SELECT
  COUNT(*) as total_business_matches,
  COUNT(CASE WHEN same_company_score > 0 THEN 1 END) as company_matches,
  COUNT(CASE WHEN same_location_score > 0 THEN 1 END) as location_matches,
  COUNT(CASE WHEN age_compatibility_score > 0 THEN 1 END) as age_matches,
  COUNT(CASE WHEN income_compatibility_score > 0 THEN 1 END) as income_matches
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_business_matches`
"""

business_stats = bq_helper.execute_query(business_count_sql)
print(
    f"📊 Business Rules Results: {business_stats.iloc[0]['total_business_matches']} total matches")
print(f"  🏢 Company matches: {business_stats.iloc[0]['company_matches']}")
print(f"  📍 Location matches: {business_stats.iloc[0]['location_matches']}")
print(f"  🎂 Age matches: {business_stats.iloc[0]['age_matches']}")
print(f"  💰 Income matches: {business_stats.iloc[0]['income_matches']}")

🔄 Running business rules matching...
✅ Business rules matching completed
📊 Business Rules Results: 40186 total matches
  🏢 Company matches: 230
  📍 Location matches: 258
  🎂 Age matches: 6017
  💰 Income matches: 9961


In [13]:
# Create Gemini 2.5 Pro model
print("\n🔄 Creating Gemini 2.5 Pro model...")
model_sql = f"""
CREATE OR REPLACE MODEL `{bq_helper.dataset_ref}.gemini_25_pro_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS(
  ENDPOINT = 'gemini-2.5-pro'
)
"""

try:
    bq_helper.execute_query(model_sql)
    print("✅ Gemini model created successfully")
except Exception as e:
    print(f"❌ Error creating model: {e}")
    print("Please ensure:")
    print("1. You have necessary permissions")
    print("2. Vertex AI API is enabled")


🔄 Creating Gemini 2.5 Pro model...
✅ Gemini model created successfully


In [14]:
# 7.5 AI Natural Language Matching
print("🤖 Running AI natural language matching...")

# Generate AI natural language matching (LIMIT 500 records for testing purposes only)
ai_sql = generate_ai_natural_language_matching_sql(
    f"{bq_helper.dataset_ref}.customers_with_embeddings",
    f"{bq_helper.dataset_ref}.gemini_25_pro_model"
)
bq_helper.execute_query(ai_sql)
print("✅ AI natural language matching completed")

# Check AI match results
ai_count_sql = f"""
SELECT
  COUNT(*) as total_ai_matches,
  AVG(ai_score) as avg_ai_score,
  AVG(confidence) as avg_confidence,
  MIN(ai_score) as min_ai_score,
  MAX(ai_score) as max_ai_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_ai_natural_language_matches`
"""

ai_stats = bq_helper.execute_query(ai_count_sql)
print(
    f"📊 AI Natural Language Results: {ai_stats.iloc[0]['total_ai_matches']} total matches")
print(f"  🤖 Avg AI score: {ai_stats.iloc[0]['avg_ai_score']:.3f}")
print(f"  🎯 Avg confidence: {ai_stats.iloc[0]['avg_confidence']:.3f}")
print(f"  📉 Min AI score: {ai_stats.iloc[0]['min_ai_score']:.3f}")
print(f"  📈 Max AI score: {ai_stats.iloc[0]['max_ai_score']:.3f}")

# Show sample AI explanations
sample_explanations_sql = f"""
SELECT
  ai_score,
  confidence,
  explanation
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_ai_natural_language_matches`
ORDER BY ai_score DESC
LIMIT 5
"""

explanations = bq_helper.execute_query(sample_explanations_sql)
print("\n🔍 Sample AI Explanations:")
for _, row in explanations.iterrows():
    print(
        f"  Score: {row['ai_score']:.3f} | Confidence: {row['confidence']:.3f}")
    print(f"  Explanation: {row['explanation']}")
    print()

🤖 Running AI natural language matching...
✅ AI natural language matching completed
📊 AI Natural Language Results: 2.0 total matches
  🤖 Avg AI score: 0.975
  🎯 Avg confidence: 0.990
  📉 Min AI score: 0.950
  📈 Max AI score: 1.000

🔍 Sample AI Explanations:
  Score: 1.000 | Confidence: 1.000
  Explanation: All fields (Name, Email, Phone, and Address) are identical in both records. The data is an exact match, leading to maximum confidence and similarity scores.

  Score: 0.950 | Confidence: 0.980
  Explanation: The name, phone number, and address are identical across both records. The only difference is the email domain ('outlook.com' vs 'example.org'), which is a minor discrepancy given the strong match in the other unique identifiers. It's highly probable these records belong to the same individual who may use multiple email accounts.



## 8. Combined Scoring and Confidence Assessment

Combine all 5 matching strategies with weighted scoring and calculate confidence levels.

In [15]:
# Combine all matching scores (now with 5 strategies)
print("🔄 Combining match scores from 5 strategies...")
combined_sql = generate_combined_scoring_sql(
    bq_helper.dataset_ref,
    "customers_with_embeddings"
)
bq_helper.execute_query(combined_sql)
print("✅ Combined scoring completed")

# Analyze combined results
analysis_sql = f"""
SELECT
  match_decision,
  confidence_level,
  COUNT(*) as count,
  AVG(combined_score) as avg_score,
  MIN(combined_score) as min_score,
  MAX(combined_score) as max_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
GROUP BY match_decision, confidence_level
ORDER BY avg_score DESC
"""

analysis_df = bq_helper.execute_query(analysis_sql)
print("\n📊 Match Decision Summary:")
display(analysis_df)

# Show top matches
top_matches_sql = f"""
SELECT
  record1_id,
  record2_id,
  source1,
  source2,
  exact_score,
  fuzzy_score,
  vector_score,
  business_score,
  ai_score,
  combined_score,
  match_decision,
  confidence_level
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
ORDER BY combined_score DESC
LIMIT 10
"""

top_matches_df = bq_helper.execute_query(top_matches_sql)
print("\n🏆 Top 10 Matches (5-Strategy Analysis):")
display(top_matches_df)

🔄 Combining match scores from 5 strategies...
✅ Combined scoring completed

📊 Match Decision Summary:


,match_decision,confidence_level,count,avg_score,min_score,max_score
0,auto_merge,high,258,0.889672,0.841038,1.00000
1,no_match,low,126,0.314257,0.300431,0.37288



🏆 Top 10 Matches (5-Strategy Analysis):


,record1_id,record2_id,source1,source2,exact_score,fuzzy_score,vector_score,business_score,ai_score,combined_score,match_decision,confidence_level
0,6f05347d-67cd-464a-9163-fbaeac666f9e,cd7cc516-9716-4941-81c0-52f9e043984b,erp,crm,1.0,1.0,1.000000,1.0,1.00,1.000000,auto_merge,high
1,97a67056-e13e-4d33-a45f-9ee7358dc9ce,d6dbf1db-f39d-4e8d-8f76-a7e6b050e2d1,crm,erp,1.0,1.0,0.977229,1.0,0.95,0.990446,auto_merge,high
2,aa795df3-fec3-4fb1-9ab1-2eeddabca382,cefe2cc7-42bc-4e25-8e20-205ace0ad4d6,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high
3,0d09d459-b177-464a-9c88-dd0237d3c57b,56893880-ea86-46a7-9645-df2339f30d6c,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high
4,4656c36a-6ff7-4881-aa98-3f42a602aa95,a30b81bf-efe7-4d23-a54f-91d0e1d6ccf0,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high
5,0d09d459-b177-464a-9c88-dd0237d3c57b,475756c0-7286-4e7c-a335-17c1dfc7c227,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high
6,728b700a-0eb4-4ce8-a60f-c646ba7ef938,e94d5a08-9e7c-499e-ad51-8ecfc8279215,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high
7,316ae334-b177-496d-8253-58be8870ea71,95ce6cdc-d285-4c83-9c20-70ec8a73b370,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high
8,453b5978-fe6f-4adb-83ef-e9c5a33a3aed,8b5bbcc1-a59d-4d05-983e-3db13a11abc5,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high
9,000c4e82-861b-4d19-b3aa-9760dcf8744e,576340a4-2b22-4884-9f19-2a6fd0d4efb4,crm,ecommerce,1.0,1.0,1.000000,1.0,0.00,0.900000,auto_merge,high


## 9. Golden Record Creation

Create master entities using survivorship rules and merge decisions.

In [16]:
# Create golden records with proper entity clustering
print("🔄 Creating golden records with transitive closure clustering...")

# Import the new function (add this to imports if needed)

# Generate and execute the golden record SQL
golden_record_sql = generate_golden_record_sql(
    bq_helper.dataset_ref,
    "customers_with_embeddings"
)

try:
    bq_helper.execute_query(golden_record_sql)
    print("✅ Golden records created successfully with proper clustering")

    # Check golden record statistics
    golden_stats_sql = f"""
    SELECT
      COUNT(*) as total_golden_records,
      AVG(source_record_count) as avg_sources_per_record,
      MAX(source_record_count) as max_sources_per_record,
      COUNT(CASE WHEN source_record_count > 1 THEN 1 END) as merged_entities,
      COUNT(CASE WHEN source_record_count = 1 THEN 1 END) as single_source_entities
    FROM `{bq_helper.dataset_ref}.golden_records`
    """

    golden_stats = bq_helper.execute_query(golden_stats_sql)
    print(f"\n📊 Golden Record Statistics:")
    print(
        f"  Total golden records: {golden_stats.iloc[0]['total_golden_records']}")
    print(
        f"  Avg sources per record: {golden_stats.iloc[0]['avg_sources_per_record']:.2f}")
    print(
        f"  Max sources per record: {golden_stats.iloc[0]['max_sources_per_record']}")
    print(f"  Merged entities: {golden_stats.iloc[0]['merged_entities']}")
    print(
        f"  Single-source entities: {golden_stats.iloc[0]['single_source_entities']}")

    # Calculate deduplication rate
    original_count = 284  # From your data generation
    golden_count = golden_stats.iloc[0]['total_golden_records']
    dedup_rate = (1 - golden_count / original_count) * 100
    print(f"\n🎯 Deduplication Results:")
    print(f"  Original records: {original_count}")
    print(f"  Golden records: {golden_count}")
    print(f"  Deduplication rate: {dedup_rate:.1f}%")

    # Show sample golden records
    sample_golden_sql = f"""
    SELECT
      master_id,
      master_name,
      master_email,
      master_phone,
      source_record_count,
      source_systems
    FROM `{bq_helper.dataset_ref}.golden_records`
    WHERE source_record_count > 1  -- Show only merged records
    ORDER BY source_record_count DESC
    LIMIT 5
    """

    sample_golden = bq_helper.execute_query(sample_golden_sql)
    if not sample_golden.empty:
        print("\n🔍 Sample Merged Golden Records:")
        display(sample_golden)

except Exception as e:
    print(f"❌ Error creating golden records: {e}")
    print("Check the SQL syntax and ensure all required tables exist")

🔄 Creating golden records with transitive closure clustering...
✅ Golden records created successfully with proper clustering

📊 Golden Record Statistics:
  Total golden records: 100.0
  Avg sources per record: 2.84
  Max sources per record: 19.0
  Merged entities: 100.0
  Single-source entities: 0.0

🎯 Deduplication Results:
  Original records: 284
  Golden records: 100.0
  Deduplication rate: 64.8%

🔍 Sample Merged Golden Records:


,master_id,master_name,master_email,master_phone,source_record_count,source_systems
0,3eb98eaf3feb34d56b52fadbd0e6c2f10a72,MICHELLE NICHOLSON,gabriel90@example.net,15659610,19,"[crm, ecommerce, erp]"
1,c2bc12d5609a5523d96f119710d08d093e77,JEFFREY WHITE,hartmanerik@outlook.com,16615533,5,"[crm, ecommerce, erp]"
2,d9822dbe3e4f99e0d619f8349fa8ccb67a1c,WENDY DAWSON,williamsjesse@example.net,457519182,5,"[crm, ecommerce]"
3,9a3522bf3b92310fa7212fc4a2188a57d0ca,LAUREN MARQUEZ,adamsnorman@example.net,765651104,5,"[crm, ecommerce, erp]"
4,73a26a02df385dcd6b880c1b766633225551,CLAUDIA RICHARDS,oscar98@gmail.com,7899611836,5,"[crm, ecommerce, erp]"


## 10. Analysis and Visualization

Analyze the MDM pipeline results and create visualizations.

In [17]:
# Create comprehensive analysis
print("📊 Analyzing MDM Pipeline Results...")

# Get overall statistics
overall_stats_sql = f"""
WITH stats AS (
  SELECT
    'Raw Records' as stage,
    COUNT(*) as record_count
  FROM `{bq_helper.dataset_ref}.raw_customers_combined`

  UNION ALL

  SELECT
    'Standardized Records' as stage,
    COUNT(*) as record_count
  FROM `{bq_helper.dataset_ref}.customers_standardized`

  UNION ALL

  SELECT
    'Records with Embeddings' as stage,
    COUNT(*) as record_count
  FROM `{bq_helper.dataset_ref}.customers_with_embeddings`
  WHERE ml_generate_embedding_result IS NOT NULL

  UNION ALL

  SELECT
    'Golden Records' as stage,
    COUNT(*) as record_count
  FROM `{bq_helper.dataset_ref}.golden_records`
)
SELECT * FROM stats ORDER BY record_count DESC
"""

overall_stats = bq_helper.execute_query(overall_stats_sql)
print("\n📈 Pipeline Statistics:")
display(overall_stats)

# Visualize pipeline flow
fig = px.funnel(
    overall_stats,
    x='record_count',
    y='stage',
    title='MDM Pipeline Data Flow',
    labels={'record_count': 'Number of Records', 'stage': 'Pipeline Stage'}
)
fig.show()

📊 Analyzing MDM Pipeline Results...

📈 Pipeline Statistics:


,stage,record_count
0,Standardized Records,284
1,Records with Embeddings,284
2,Raw Records,284
3,Golden Records,100


In [18]:
# Analyze 5-strategy matching effectiveness
strategy_analysis_sql = f"""
SELECT
  'Exact Matching' as strategy,
  COUNT(*) as matches_found,
  AVG(exact_score) as avg_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
WHERE exact_score > 0

UNION ALL

SELECT
  'Fuzzy Matching' as strategy,
  COUNT(*) as matches_found,
  AVG(fuzzy_score) as avg_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
WHERE fuzzy_score > 0

UNION ALL

SELECT
  'Vector Matching' as strategy,
  COUNT(*) as matches_found,
  AVG(vector_score) as avg_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
WHERE vector_score > 0

UNION ALL

SELECT
  'Business Rules' as strategy,
  COUNT(*) as matches_found,
  AVG(business_score) as avg_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
WHERE business_score > 0

UNION ALL

SELECT
  'AI Natural Language' as strategy,
  COUNT(*) as matches_found,
  AVG(ai_score) as avg_score
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
WHERE ai_score > 0

ORDER BY matches_found DESC
"""

strategy_stats = bq_helper.execute_query(strategy_analysis_sql)
print("\n🎯 5-Strategy Matching Effectiveness:")
display(strategy_stats)

# Create visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Matches Found by Strategy', 'Average Score by Strategy'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

fig.add_trace(
    go.Bar(x=strategy_stats['strategy'],
           y=strategy_stats['matches_found'], name='Matches'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=strategy_stats['strategy'],
           y=strategy_stats['avg_score'], name='Avg Score'),
    row=1, col=2
)

fig.update_layout(title_text="5-Strategy Matching Analysis", showlegend=False)
fig.show()


🎯 5-Strategy Matching Effectiveness:


,strategy,matches_found,avg_score
0,Vector Matching,384,0.934118
1,Fuzzy Matching,384,0.832410
2,Business Rules,340,0.782059
3,Exact Matching,258,1.000000
4,AI Natural Language,2,0.975000


In [19]:
# Analyze confidence distribution
confidence_dist_sql = f"""
SELECT
  ROUND(combined_score, 1) as score_bucket,
  COUNT(*) as count,
  match_decision
FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
GROUP BY score_bucket, match_decision
ORDER BY score_bucket
"""

confidence_dist = bq_helper.execute_query(confidence_dist_sql)
print("\n📊 Confidence Score Distribution:")
display(confidence_dist.head(10))

# Create confidence distribution plot
fig = px.histogram(
    confidence_dist,
    x='score_bucket',
    y='count',
    color='match_decision',
    title='Distribution of Match Confidence Scores (5-Strategy)',
    labels={'score_bucket': 'Confidence Score', 'count': 'Number of Matches'}
)
fig.show()


📊 Confidence Score Distribution:


,score_bucket,count,match_decision
0,0.3,120,no_match
1,0.4,6,no_match
2,0.8,20,auto_merge
3,0.9,236,auto_merge
4,1.0,2,auto_merge


## 11. Performance Metrics and Summary

Calculate key performance indicators for the 5-strategy MDM pipeline.

In [20]:
# Calculate key metrics
print("📈 Calculating 5-Strategy MDM Performance Metrics...")

# Data quality metrics
quality_metrics_sql = f"""
WITH quality_stats AS (
  SELECT
    COUNT(*) as total_records,
    COUNT(CASE WHEN email_clean IS NOT NULL THEN 1 END) / COUNT(*) as email_completeness,
    COUNT(CASE WHEN phone_clean IS NOT NULL THEN 1 END) / COUNT(*) as phone_completeness,
    COUNT(CASE WHEN address_clean IS NOT NULL THEN 1 END) / COUNT(*) as address_completeness,
    COUNT(DISTINCT email_clean) / COUNT(CASE WHEN email_clean IS NOT NULL THEN 1 END) as email_uniqueness,
    COUNT(DISTINCT phone_clean) / COUNT(CASE WHEN phone_clean IS NOT NULL THEN 1 END) as phone_uniqueness
  FROM `{bq_helper.dataset_ref}.customers_standardized`
)
SELECT
  total_records,
  ROUND(email_completeness * 100, 2) as email_completeness_pct,
  ROUND(phone_completeness * 100, 2) as phone_completeness_pct,
  ROUND(address_completeness * 100, 2) as address_completeness_pct,
  ROUND(email_uniqueness * 100, 2) as email_uniqueness_pct,
  ROUND(phone_uniqueness * 100, 2) as phone_uniqueness_pct
FROM quality_stats
"""

quality_metrics = bq_helper.execute_query(quality_metrics_sql)
print("\n📊 Data Quality Metrics:")
display(quality_metrics)

# Matching effectiveness
matching_metrics_sql = f"""
WITH matching_stats AS (
  SELECT
    COUNT(*) as total_potential_matches,
    COUNT(CASE WHEN match_decision = 'auto_merge' THEN 1 END) as auto_merge_count,
    COUNT(CASE WHEN match_decision = 'human_review' THEN 1 END) as human_review_count,
    COUNT(CASE WHEN match_decision = 'no_match' THEN 1 END) as no_match_count,
    AVG(combined_score) as avg_combined_score
  FROM `{bq_helper.dataset_ref}.customers_with_embeddings_combined_matches`
)
SELECT
  total_potential_matches,
  auto_merge_count,
  human_review_count,
  no_match_count,
  ROUND(auto_merge_count / total_potential_matches * 100, 2) as auto_merge_rate_pct,
  ROUND(human_review_count / total_potential_matches * 100, 2) as human_review_rate_pct,
  ROUND(avg_combined_score, 3) as avg_combined_score
FROM matching_stats
"""

matching_metrics = bq_helper.execute_query(matching_metrics_sql)
print("\n🎯 5-Strategy Matching Effectiveness:")
display(matching_metrics)

📈 Calculating 5-Strategy MDM Performance Metrics...

📊 Data Quality Metrics:


,total_records,email_completeness_pct,phone_completeness_pct,address_completeness_pct,email_uniqueness_pct,phone_uniqueness_pct
0,284,100.0,96.13,100.0,60.92,44.32



🎯 5-Strategy Matching Effectiveness:


,total_potential_matches,auto_merge_count,human_review_count,no_match_count,auto_merge_rate_pct,human_review_rate_pct,avg_combined_score
0,384,258,0,126,67.19,0.0,0.701


In [21]:
# Final summary
print("\n" + "="*60)
print("🎉 5-STRATEGY MDM PIPELINE EXECUTION SUMMARY")
print("="*60)

print(f"\n📊 DATA PROCESSING:")
print(
    f"  • Generated {total_records:,} sample records from {len(datasets)} sources")
print(f"  • Representing {generator.num_unique_customers:,} unique customers")
print(
    f"  • Duplication factor: {total_records / generator.num_unique_customers:.2f}x")

if not quality_metrics.empty:
    print(f"\n📈 DATA QUALITY:")
    print(
        f"  • Email completeness: {quality_metrics.iloc[0]['email_completeness_pct']:.1f}%")
    print(
        f"  • Phone completeness: {quality_metrics.iloc[0]['phone_completeness_pct']:.1f}%")
    print(
        f"  • Address completeness: {quality_metrics.iloc[0]['address_completeness_pct']:.1f}%")

if not matching_metrics.empty:
    print(f"\n🎯 5-STRATEGY MATCHING RESULTS:")
    print(
        f"  • Total potential matches: {matching_metrics.iloc[0]['total_potential_matches']:,}")
    print(
        f"  • Auto-merge rate: {matching_metrics.iloc[0]['auto_merge_rate_pct']:.1f}%")
    print(
        f"  • Human review rate: {matching_metrics.iloc[0]['human_review_rate_pct']:.1f}%")
    print(
        f"  • Average combined score: {matching_metrics.iloc[0]['avg_combined_score']:.3f}")

print(f"\n🏗️ ENHANCED ARCHITECTURE HIGHLIGHTS:")
print(f"  • 100% BigQuery-native implementation")
print(f"  • Latest gemini-embedding-001 model for vector matching")
print(f"  • NEW: Gemini 2.5 Pro for AI natural language matching")
print(f"  • Vector indexes for fast similarity search")
print(f"  • 5-strategy matching (exact, fuzzy, vector, rules, AI)")
print(f"  • Enhanced weighted ensemble scoring")
print(f"  • AI-powered explanations for match decisions")
print(f"  • Automated confidence scoring and decision making")
print(f"  • Survivorship rules for golden record creation")

print(f"\n✅ 5-STRATEGY PIPELINE COMPLETED SUCCESSFULLY!")
print("="*60)


🎉 5-STRATEGY MDM PIPELINE EXECUTION SUMMARY

📊 DATA PROCESSING:
  • Generated 284 sample records from 3 sources
  • Representing 120 unique customers
  • Duplication factor: 2.37x

📈 DATA QUALITY:
  • Email completeness: 100.0%
  • Phone completeness: 96.1%
  • Address completeness: 100.0%

🎯 5-STRATEGY MATCHING RESULTS:
  • Total potential matches: 384.0
  • Auto-merge rate: 67.2%
  • Human review rate: 0.0%
  • Average combined score: 0.701

🏗️ ENHANCED ARCHITECTURE HIGHLIGHTS:
  • 100% BigQuery-native implementation
  • Latest gemini-embedding-001 model for vector matching
  • NEW: Gemini 2.5 Pro for AI natural language matching
  • Vector indexes for fast similarity search
  • 5-strategy matching (exact, fuzzy, vector, rules, AI)
  • Enhanced weighted ensemble scoring
  • AI-powered explanations for match decisions
  • Automated confidence scoring and decision making
  • Survivorship rules for golden record creation

✅ 5-STRATEGY PIPELINE COMPLETED SUCCESSFULLY!
